## CS109a Final Project - Spotify Recommendation System
Group \#55: Nick Kochanek, Jack Connolly, Chris Jarrett, Andrew Soldini

**Project Goal:**  
Our goal for this project was to develop a system that could recommend reasonable songs to continue a playlist give a certain number of "seed" tracks. We formalized this task as giving a model a list of $K$ input songs (as Spotify uris) and having the model output $500$ suggested uris (ideally ranked by relevance). This falls in line with the formal specifications for the Spotify RecSys challenge, and allows us to compare results and approaches with top teams there. As such, we decided to evaluate our models using the same metrics the contest was based on, which will be described further on.

In [2]:
import requests

from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

### Data and EDA
The data sources we utilized in our explorations were the Million Playlist Dataset and the Spotify API. The MPD gave us the essential data to train models on subsets of playlists and evaluate the accuracy of our predictions. We used to Spotify API to get pre-computed audio features for songs, which allowed us to expore alternative model choices more in line with what we looked at in class. 

In [ ]:
# Some EDA code, generate some cool graphs

In [ ]:
# More graphs and charts, both for MPD and for Spotify audio features

## Models

We explored a variety of different models and model types, but because the style of problem was different than those we studied in class, we were forced to explore different techniques than those we had seen. Our baseline models use a combination of KMeans Clustering and/or KNN, while the top two performing models are a Markov Walk on a Network and Collaborative Filtering.

### Baseline: Song Based KMeans Clustering and KNN

In [ ]:
# Song-Based KMeans code here

### Another baseline: Playlist Based KNN

In [ ]:
# Playlist Based KNN (Chris)

### Collaborative Filtering

In [3]:
# Filtering code here (Chris)

### Network Based Markov Model
This model is a probabilistic one that builds up a network where each vertex represents a song and each edge represents two song sharing a playlist (where more shared playlists leads to higher weighting). Then for prediction, for each of the input $K$ songs, many one step random walks are taken, and the most popular songs that show up in these walks are then returned as a list of 500 song recommendations (after getting rid of duplicates that are already in the playlist). This model consistently performed the best of our models, although building up a large network takes a significant amount of time and space.

In [4]:
# Network Building code (Nick)

## Evaluation
We decided to evaluate our models based on the same metrics used in the Spotify RecSys [contest rules](https://recsys-challenge.spotify.com/rules), namely R-Precision (RPrec), Normalized Discounted Cumulative Gain (NDCG), and Recommended Song Clicks (RSC). In the following definitions, $G$ is the set of ground truth tracks representing the held out songs from each playlist and $R$ is the ordered list of recommended songs returned by the recommendation system.

* R-Precision: The metric counts "number of retrieved relevant tracks divided by the number of known relevant tracks," rewarding the total number of retrieved relevant tracks, regardless of order.
$$\text{R-precision} = \frac{\left| G \cap R_{1:|G|} \right|}{|G|}$$

* Normalized Discounted Cumulative Gain (NDCG): This metric takes into account the order of the returned songs, rewarding relevant songs placed higher in the returned list. It is calculated as Discounted Cumulative Gain (DCG), divided by the Ideal Discounted Cumulative Gain (IDCG), where the returned songs are ordered perfectly. That calculation looks like:
$$DCG = rel_1 + \sum_{i=2}^{|R|} \frac{rel_i}{\log_2 (i + 1)}$$
$$IDCG = 1 + \sum_{i=2}^{|G|} \frac{1}{\log_2 (i + 1)}$$
$$NDCG = \frac{DCG}{IDCG}$$

* Recommended Songs Clicks (RSC): This measures how many "clicks" a Spotify user would need to find the first relevant song in the recommendations (the first song actually in the rest of the playlist $G$), where Spotify displays recommended songs in groups of 10. Therefore it's simply finding the first relevant song and returning its position in the list divided by 10 and truncated. Or more formally:
$$\text{clicks} = \left\lfloor \frac{ \arg\min_i \{ R_i\colon R_i \in G|\} - 1}{10} \right\rfloor$$

We have implemented these metrics in code below:

In [5]:
from math import log2

class Evaluator():
    """Superclass for evaluation functions"""
    
    def __init__(self, name):
        self.name = name
        
    def evaluate(self, output, expected):
        """
        Output will be the output of the model for some list of playlists
        - Shape of (# playlists, 500)

        Expected will be the held out songs from each playlist
        - List of lists of various sizes

        Note: Each "song" will be the unique spotify uri of a song
        """
        raise NotImplementedError

        
class RPrecision(Evaluator):
    """
    R-precision measures the number of held out songs correctly 
        retrieved by the model output 
    """
    def __init__(self):
        Evaluator.__init__(self, 'R-Precision')
        
    def evaluate(self, output, expected, return_all=False):
    
        def rprec_one(output_, expected_):
            expected_size = len(expected_)
            common_set = set(output_).intersection(set(expected_))
            common_size = len(common_set)
            if expected_size == 0 or common_size == 0:
                return 0.0
            return common_size / expected_size
        
        r_precs = [rprec_one(out, exp) for (out, exp) in zip(output, expected)]
        if return_all:
            return np.mean(r_precs), r_precs
        return np.mean(r_precs)

    
class NDCG(Evaluator):
    """
    Normalized discounted cumulative gain also takes into 
        account how the system ordered the suggestions
    """
    def __init__(self):
        Evaluator.__init__(self, 'NDCG')
        
    def evaluate(self, output, expected):
        
        def ndcg_one(output_, expected_):
            dcg, idcg = 0.0, 0.0
            
            if len(output_) == 0 or len(expected_) == 0:
                return 0.0
            
            expected_ = set(expected_)
            for i in range(len(output_)):
                # Prediction DCG
                if output_[i] in expected_:
                    if i == 0:
                        dcg += 1.0
                    else:
                        dcg += 1.0 / log2(i + 2.0)

                if i < len(expected_):
                    if i == 0:
                        idcg += 1.0
                    else:
                        idcg += 1.0 / log2(i + 2.0)
            
            return dcg / idcg
        
        return np.mean([ndcg_one(out, exp) for (out, exp) in zip(output, expected)])
        
        
class RSC(Evaluator):
    """
    Recommended Song Clicks measures how many times a user
    would have to click through the suggestions to find a song that 
    was a ground truth song
    """
    def __init__(self):
        Evaluator.__init__(self, 'RSC')
        
    def evaluate(self, output, expected):
        
        def rsc_one(output_, expected_):
            if len(output_) == 0 or len(expected_) == 0:
                return 51
            
            output_len = len(output_)
            expected_ = set(expected_)
            for i in range(output_len):
                if output_[i] in expected_:
                    return i//10
            return 51
        
        return np.mean([rsc_one(out, exp) for (out, exp) in zip(output, expected)])

In [6]:
def evaluate_model(output, expected):
    r_prec = RPrecision().evaluate(output, expected)
    ndcg = NDCG().evaluate(output, expected)
    rsc = RSC().evaluate(output, expected)
    print(f"R-Precision: {r_prec}, NCDG: {ndcg}, RSC: {rsc}")

**Evaluation of each model:**

In [7]:
# for model in models:
#     output = model.predict(X_test)
#     evaluate_model(output, expected)

### Conclusions and Interpretations
Overall, we have seen that the filtering and network models perform the best, significantly improving over the baseline models using nearest neighbor techniques. Our final models were comparable with some of the top models in the RecSys challenge, so we are very satisfied with our results. If we had more time and computing power, we would have liked to scale both of those models up larger, as they were both limited in terms of their size (the network was trained on about 14000 playlists and ended up being about 7GB while filtering was only able to handle about **HOW MANY PLAYLISTS**). Ideally, we would be able to utilize sklearn's suppoer for sparse matrices to scale up filtering, but we weren't able to finalize that. 

Music recommendation in general is a challenging problem, with millions of songs to choose from and a large variety of songs within. More complex techniques like deep RNNs and autoencoders seemed attractive at the beginning of the project, but ultimately weren't feasible for us to complete. This forced us to adapt and implement the fairly different models seen here. Overall, we feel confident in our model's ability to find relevant songs to continue and put together a great playlist.